In [1]:
from PIL import Image
from rl_benchmarks.models import iBOTViT
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
import pathlib
from tqdm import tqdm
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
from multiprocessing import Pool
import umap
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from rl_benchmarks.utils.linear_evaluation import get_binary_class_metrics, get_bootstrapped_metrics

from PIL import Image
import pathlib
from tqdm import tqdm
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
from multiprocessing import Pool
# import umap
import numpy as np
import matplotlib.pyplot as plt
import sklearn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

from sklearn.metrics import f1_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, cohen_kappa_score, accuracy_score
import torch.nn.functional as F
import sys
import time

import shutil
import os

# import albumentations as A
# from albumentations.pytorch import ToTensorV2
import cv2
import time
from sklearn.preprocessing import label_binarize

import timm
from metrics import report

In [2]:
import sys
sys.path.append('/home/yuxin/bme/BCaCAD/model')
from patch_based_test.img import QiLuROI

In [3]:
class IBOTMultiTaskModel(nn.Module):
    def __init__(self, num_classes):
        super(IBOTMultiTaskModel, self).__init__()
        weights_path = '/home/yuxin/Downloads/ibot_vit_base_pancan.pth'
        self.base_model = iBOTViT(architecture="vit_base_pancan", encoder="teacher", weights_path=weights_path)
        # print(self.base_model)
        self.num_features = 768
        self.num_classes = num_classes

        if isinstance(num_classes, list):
            self.heads = nn.ModuleList([nn.Linear(self.num_features, num_class) for num_class in num_classes])
        else:
            self.head = self.base_model.head
            self.head.fc = nn.Linear(self.num_features, num_classes)

    def forward(self, x):
        # Forward pass through the base model
        x = self.base_model(x)
        if isinstance(self.num_classes, list):
            x = [head(x) for head in self.heads]
        else:
            x = self.head(x)
        return x

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
num_classes = [3,3]
img_size = patch_size = 384
to_tensor= transforms.Compose([
            transforms.Resize((img_size,img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


In [5]:
model = IBOTMultiTaskModel(num_classes)
device = "cuda"
model.to(device)
model = nn.DataParallel(model)
weight_path = '/mnt/hd0/project/bcacad/model/pretrainSSL_ibot_vit+ibot_ft+fsl_ft/model-5.pth'
model.load_state_dict(torch.load(weight_path, map_location=device)['model_state_dict'])


2024-12-01 08:05:05.765 | INFO     | rl_benchmarks.models.feature_extractors.ibot_vit:__init__:78 - Pretrained weights found at /home/yuxin/Downloads/ibot_vit_base_pancan.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v', 'head.last_layer2.weight_g', 'head.last_layer2.weight_v'])


<All keys matched successfully>

In [6]:
def to_feature(model, device, path):
    size = 336
    bs = 8
    im = QiLuROI(str(path), 10, 10, size)
    im.setIterator(size)
    patches = [to_tensor(p) for p in im]
    for i in range(0, len(patches), bs):
        x = torch.stack(patches[i:i+bs], dim=0)
        x = x.to(device)
        y = model(x)
        if i == 0:
            features = y.detach().cpu().numpy()
        else:
            features = np.concatenate([features, y.detach().cpu().numpy()], axis=0)
    return features


In [7]:
from pathlib import Path
data_root = Path('/mnt/hd0/project/bcacad/data/roi-level')
save_root = Path('/mnt/hd0/project/bcacad/model/roi_features')
# test_cohorts = ['qduh', 'shsu', 'bracs', 'bcnb', 'bach', 'apght']
# test_cohorts = ['bcnb']
test_cohorts = ['bjszhp']

In [8]:
for cohort in test_cohorts:
    print(cohort)
    src_dir = data_root / cohort
    save_dir = save_root / cohort
    im_files = list(src_dir.glob('**/*.*'))
    for im_file in im_files:
        feature = to_feature(model.module.base_model, device, im_file)
        # label = im_file.parent.name
        save_path = save_dir / 'test' /f'{im_file.stem}.npy'
        save_path.parent.mkdir(parents=True, exist_ok=True)
        np.save(save_path, feature)
    

bjszhp


In [7]:
# for 100 roi test
from pathlib import Path
# data_root = Path('/mnt/hd0/project/bcacad/data/100_test_roi')
data_root = Path('/mnt/hd0/project/bcacad/data/roi-level/100_rois')
# save_root = Path('/mnt/hd0/project/bcacad/model/100_test_roi')
save_root = Path('/mnt/hd0/project/bcacad/model/100_rois')
im_files = list(data_root.glob('**/*.*'))
for im_file in im_files:
    feature = to_feature(model.module.base_model, device, im_file)
    save_path = save_root /f'{im_file.stem}.npy'
    save_path.parent.mkdir(parents=True, exist_ok=True)
    np.save(save_path, feature)

In [2]:
wsi_excel_path = '/mnt/hd0/project/bcacad/data/high-1_wsi.xlsx'
import pandas as pd
import shutil
from pathlib import Path
df = pd.read_excel(wsi_excel_path)
wsi_names = df['name'].to_list()
roi_root = Path('/mnt/hd0/project/bcacad/model/100_rois')
feature_files= list(roi_root.glob('*.npy'))
save_wsi_root = Path('/mnt/hd0/project/bcacad/model/100_wsi')
for wsi_name in wsi_names:
    wsi_feature = []
    for feature_file in feature_files:
        if wsi_name in feature_file.stem:
            wsi_feature.append(feature_file)
    save_root = save_wsi_root / wsi_name
    save_root.mkdir(parents=True, exist_ok=True)
    #copy
    for f in wsi_feature:
        shutil.copy(f, save_root)
